# Amazon SKU Tracker

In [ ]:
import pandas as pd
from selenium import webdriver
import warnings
warnings.filterwarnings("ignore")
from time import sleep
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime
from selenium.webdriver.chrome.options import Options

In [ ]:
pincodes = pd.read_excel("asin-pin.xlsx",sheet_name = "pincodes")
asin = pd.read_excel("asin-pin.xlsx",sheet_name = "asin")

In [ ]:
asin = asin[:10]

In [ ]:
a = pincodes.copy()
for i in range(len(asin)-1):
     a = pd.concat([a,pincodes])

b = asin.copy()
for i in range(len(pincodes)-1):
    b = pd.concat([b,asin])

In [ ]:
b.sort_index(inplace=True)
a["ASIN"] = list(b["ASIN"])

In [ ]:
a

In [ ]:
pincode = list(a["Pincode"])
asin = list(a["ASIN"])
city = list(a["City"])

In [ ]:
path = r"C:\Users\Sanchit\Downloads\chromedriver.exe"
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')  
options.add_argument('--no-sandbox')
options.add_argument('window-size=1920x1080')

In [ ]:
count = 0
df = {"Date":[],"Pincode":[],"City":[],"ASIN":[],"Selling Price":[],"Availability":[],"Star Rating":[],
      "Seller":[],"Delivery":[],"No. of Ratings":[],"No. of Reviews":[]}

In [ ]:
for i in range(154,len(asin)):
    browser = webdriver.Chrome(executable_path=path, chrome_options = options)
    browser.get("https://www.amazon.in//dp/"+asin[i])
    
    pincode_button = browser.find_element_by_id('glow-ingress-line2')
    pincode_button.click()
    sleep(3)
    pincode_key = browser.find_element_by_id('GLUXZipUpdateInput')
    pincode_key.send_keys(pincode[i])
    pincode_apply = browser.find_element_by_id('GLUXZipUpdate')
    pincode_apply.click()
    sleep(3)
        
    if browser.find_element_by_id('availability').text != 'Currently Unavailable.':
        
        
        try : 
            price = browser.find_element_by_id("priceblock_ourprice").text
            price = price.replace('₹','')
            price = float(price)
            df["Selling Price"].append(price)
       
        except NoSuchElementException:
            try:
                price = browser.find_element_by_id("priceblock_dealprice").text
                price = price.replace('₹','')
                price = float(price)
                df["Selling Price"].append(price)
                
            except NoSuchElementException:
                try:
                    price = browser.find_element_by_xpath("//*[@id='corePrice_feature_div']/div/span[1]/span[2]").text
                    price = price.replace('₹','')
                    price = price.replace('\n00','')
                    price = float(price)
                    df["Selling Price"].append(price)
                
                except NoSuchElementException:
                    price = browser.find_element_by_xpath('//*[@id="corePrice_feature_div"]/div/span[1]/span[2]/span[2]').text
                    price = price.replace('₹','')
                    price = float(price)
                    df["Selling Price"].append(price)

        stars = browser.find_element_by_id('acrPopover').get_attribute('title')
        stars = stars[:3]
        stars = float(stars)
        df["Star Rating"].append(stars)

        sell = browser.find_element_by_xpath('//*[@id="merchant-info"]/a[1]/span').text
        df["Seller"].append(sell)
        
        try:
            delivery_time =  browser.find_element_by_xpath('//*[@id="mir-layout-DELIVERY_BLOCK-slot-DELIVERY_MESSAGE"]/b').text
            df["Delivery"].append(delivery_time)
        except NoSuchElementException:
            df["Delivery"].append("NA")

        reviews_button = browser.find_element_by_xpath('//a[@data-hook="see-all-reviews-link-foot"]')
        sleep(1)
        reviews_button.click()
        sleep(1)

        no_review_ratings = browser.find_element_by_xpath('//div[@data-hook="cr-filter-info-review-rating-count"]').text
        x = no_review_ratings.split(' | ')
        a = x[0]
        a = a.replace(" global ratings",'')
        a = a.replace(" total ratings",'')
        a = a.replace(',','')
        try:
            a = int(a)
            df["No. of Ratings"].append(a)
            b = x[1]
            b = b.replace(" global reviews",'')
            b = b.replace(" with reviews",'')
            b = b.replace(',','')
            b = int(b)
            df['No. of Reviews'].append(b)
        except ValueError:
            z = a.split(' ')
            df["No. of Ratings"].append(int(z[0]))
            df["No. of Reviews"].append(int(z[1]))
            
        df["Availability"].append('In Stock.')

       
        
    else:
        df["Availability"].append('Out of Stock.')
        df["Selling Price"].append("NA")
        df["Star Rating"].append("NA")
        df["Seller"].append("NA")
        df["Delivery"].append("NA")
        df["Fastest Delivery By"].append("NA")
        df["No. of Ratings"].append("NA")
        df["No. of Reviews"].append("NA")
    
    df["ASIN"].append(asin[i])
    df["Pincode"].append(pincode[i])
    df["City"].append(city[i])
    
    now = datetime.now()
    df["Date"].append(now)
    
    count+=1
    print(count)
    browser.close()

In [ ]:
for i in df.keys():
    print(i)
    print(len(df[i]))

In [ ]:
# df["Availability"].pop(267)
# df["Selling Price"].pop(267)
# df["Star Rating"].pop(267)
# df["Delivery"].pop(267)
# df["Seller"].pop(267)

In [ ]:
required = pd.DataFrame(df)

In [ ]:
required

In [ ]:
required["Seller"].unique()

In [ ]:
required["Seller"].replace({'':'Yogabars'},inplace=True)

In [ ]:
required['Delivery'].unique()

In [ ]:
required['Delivery'].replace({'':'NA'},inplace=True)

In [ ]:
required.to_excel('amz_fin_sheet1.xlsx')